In [1]:
import numpy as np
import pandas as pd
import mne
from mne.preprocessing import ICA
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os


def load_eeg_data(data_dir, metadata_path):
    """
    Load EEG data from CSV files and convert to MNE Raw objects.
    """
    # Load metadata
    metadata = pd.read_csv(metadata_path)
    
    # Standard 10-20 montage
    montage = mne.channels.make_standard_montage('standard_1020')
    
    raw_objects = []
    for _, row in metadata.iterrows():
        # Load data
        filepath = os.path.join(data_dir, row['filename'])
        df = pd.read_csv(filepath)
        
        # Extract time and data
        time = df['time'].values
        data = df.drop('time', axis=1).values.T
        
        # Create MNE Raw object
        ch_names = df.columns[:-1].tolist()  # Exclude 'time' column
        info = mne.create_info(ch_names, sfreq=int(1/(time[1]-time[0])), ch_types='eeg')
        raw = mne.io.RawArray(data, info)
        raw.set_montage(montage)
        
        raw_objects.append((raw, row['gender']))
    
    return raw_objects

def preprocess_data(raw_objects):
    """
    Preprocess EEG data using ICA.
    """
    processed_data = []
    
    for raw, gender in raw_objects:
        # Apply band-pass filter
        raw_filtered = raw.copy().filter(l_freq=1, h_freq=40)
        
        # Apply ICA
        ica = ICA(n_components=15, random_state=42)
        ica.fit(raw_filtered)
        
        # Apply ICA to remove artifacts
        raw_cleaned = raw_filtered.copy()
        ica.apply(raw_cleaned)
        
        processed_data.append((raw_cleaned, gender))
    
    return processed_data

def create_topographic_plots(processed_data):
    """
    Create interactive topographic maps comparing male and female brain activity.
    """
    # Separate male and female data
    male_data = [data for data, gender in processed_data if gender == 'male']
    female_data = [data for data, gender in processed_data if gender == 'female']
    
    # Calculate average power for each group
    male_power = np.mean([data.get_data()**2 for data in male_data], axis=0)
    female_power = np.mean([data.get_data()**2 for data in female_data], axis=0)
    
    # Get channel positions
    pos = male_data[0].info.get_montage().get_positions()['ch_pos']
    ch_names = male_data[0].info.ch_names
    
    # Create x, y coordinates for plotting
    x = [pos[ch][0] for ch in ch_names]
    y = [pos[ch][1] for ch in ch_names]
    
    # Create figures for male and female data
    fig_male = go.Figure()
    fig_female = go.Figure()
    
    # Add scatter plots for electrode positions
    fig_male.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers+text',
        marker=dict(size=10, color=np.mean(male_power, axis=1), colorscale='Viridis',
                   showscale=True, colorbar=dict(title='Power (μV²)')),
        text=ch_names,
        textposition="top center",
        name='Electrodes'
    ))
    
    fig_female.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers+text',
        marker=dict(size=10, color=np.mean(female_power, axis=1), colorscale='Viridis',
                   showscale=True, colorbar=dict(title='Power (μV²)')),
        text=ch_names,
        textposition="top center",
        name='Electrodes'
    ))
    
    # Update layouts
    for fig, title in [(fig_male, 'Male Brain Activity During Financial Decision Making'),
                      (fig_female, 'Female Brain Activity During Financial Decision Making')]:
        fig.update_layout(
            title=dict(
                text=title,
                x=0.5,
                y=0.95,
                xanchor='center',
                yanchor='top',
                font=dict(size=20)
            ),
            showlegend=False,
            xaxis=dict(title='Left - Right', showgrid=False),
            yaxis=dict(title='Posterior - Anterior', showgrid=False),
            width=800,
            height=800
        )
    
    return fig_male, fig_female

def create_time_frequency_plots(processed_data):
    """
    Create interactive time-frequency plots using Plotly.
    """
    # Separate male and female data
    male_data = np.mean([data.get_data() for data, gender in processed_data if gender == 'male'], axis=0)
    female_data = np.mean([data.get_data() for data, gender in processed_data if gender == 'female'], axis=0)
    
    # Create time vector
    time = np.linspace(0, male_data.shape[1]/256, male_data.shape[1])
    
    # Create subplot figure
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Male Brain Activity Over Time', 'Female Brain Activity Over Time'),
        vertical_spacing=0.15
    )
    
    # Add male data trace
    fig.add_trace(
        go.Heatmap(
            z=male_data,
            x=time,
            y=processed_data[0][0].ch_names,
            colorscale='Viridis',
            colorbar=dict(title='Amplitude (μV)', y=0.85, len=0.35),
            hoverongaps=False,
            name='Male'
        ),
        row=1, col=1
    )
    
    # Add female data trace
    fig.add_trace(
        go.Heatmap(
            z=female_data,
            x=time,
            y=processed_data[0][0].ch_names,
            colorscale='Viridis',
            colorbar=dict(title='Amplitude (μV)', y=0.35, len=0.35),
            hoverongaps=False,
            name='Female'
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title=dict(
            text='Brain Activity During Financial Decision Making',
            x=0.5,
            y=0.95,
            xanchor='center',
            yanchor='top',
            font=dict(size=20)
        ),
        xaxis_title='Time (s)',
        xaxis2_title='Time (s)',
        yaxis_title='EEG Channels',
        yaxis2_title='EEG Channels',
        height=1000,
        width=1200,
        showlegend=False
    )
    
    return fig

def save_plots_to_html(processed_data, output_dir='eeg_visualizations'):
    """
    Save all plots to HTML files.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Create and save topographic plots
    fig_male_topo, fig_female_topo = create_topographic_plots(processed_data)
    fig_male_topo.write_html(os.path.join(output_dir, 'male_topographic.html'))
    fig_female_topo.write_html(os.path.join(output_dir, 'female_topographic.html'))
    
    # Create and save time-frequency plots
    fig_time_freq = create_time_frequency_plots(processed_data)
    fig_time_freq.write_html(os.path.join(output_dir, 'time_frequency_comparison.html'))
    
    print(f"Plots have been saved to the '{output_dir}' directory:")
    print("1. male_topographic.html - Male brain activity topographic map")
    print("2. female_topographic.html - Female brain activity topographic map")
    print("3. time_frequency_comparison.html - Time-frequency comparison of both groups")


def generate_3d_brain_surface():
    """
    Generate a simplified 3D brain surface using parametric equations.
    """
    # Create parametric coordinates for a brain-like shape
    u = np.linspace(0, 2*np.pi, 100)
    v = np.linspace(-np.pi/2, np.pi/2, 100)
    u, v = np.meshgrid(u, v)
    
    # Create brain shape using modified ellipsoid equations
    x = 50 * np.cos(v) * np.cos(u)
    y = 40 * np.cos(v) * np.sin(u)
    z = 30 * np.sin(v)
    
    # Add some asymmetry and irregularity to make it more brain-like
    x += 5 * np.sin(2*u) * np.cos(3*v)
    y += 5 * np.sin(3*u) * np.cos(2*v)
    z += 5 * np.sin(4*v)
    
    return x, y, z


def map_eeg_to_3d_coordinates(ch_pos):
    """
    Map EEG channel positions to 3D surface coordinates.
    """
    # Extract 2D positions
    x = np.array([pos[0] for pos in ch_pos.values()])
    y = np.array([pos[1] for pos in ch_pos.values()])
    
    # Convert from 2D to 3D coordinates on the brain surface
    # Scale and adjust positions to fit the brain surface
    scale = 50
    x_3d = x * scale
    y_3d = y * scale
    
    # Calculate z coordinate based on x and y (project onto a sphere)
    r = scale
    z_3d = np.sqrt(r**2 - x_3d**2 - y_3d**2)
    z_3d = np.real(z_3d)  # Take real part in case of numerical errors
    
    return x_3d, y_3d, z_3d


def create_3d_brain_plots(processed_data):
    """
    Create interactive 3D brain plots showing EEG activity.
    """
    # Get channel positions
    ch_pos = processed_data[0][0].info.get_montage().get_positions()['ch_pos']
    ch_names = processed_data[0][0].info.ch_names
    
    # Separate male and female data
    male_data = [data for data, gender in processed_data if gender == 'male']
    female_data = [data for data, gender in processed_data if gender == 'female']
    
    # Calculate average power for each group
    male_power = np.mean([np.mean(data.get_data()**2, axis=1) for data in male_data], axis=0)
    female_power = np.mean([np.mean(data.get_data()**2, axis=1) for data in female_data], axis=0)
    
    # Generate brain surface
    x_brain, y_brain, z_brain = generate_3d_brain_surface()
    
    # Map EEG positions to 3D coordinates
    x_eeg, y_eeg, z_eeg = map_eeg_to_3d_coordinates(ch_pos)
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scene'}, {'type': 'scene'}]],
        subplot_titles=('Male Brain Activity', 'Female Brain Activity')
    )
    
    # Add brain surface for male plot
    fig.add_trace(
        go.Surface(
            x=x_brain, y=y_brain, z=z_brain,
            opacity=0.8,
            colorscale='Greys',
            showscale=False,
            name='Brain Surface'
        ),
        row=1, col=1
    )
    
    # Add EEG points for male plot
    fig.add_trace(
        go.Scatter3d(
            x=x_eeg, y=y_eeg, z=z_eeg,
            mode='markers+text',
            marker=dict(
                size=10,
                color=male_power,
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(
                    title='Power (μV²)',
                    x=0.45
                )
            ),
            text=ch_names,
            textposition="top center",
            name='Male EEG'
        ),
        row=1, col=1
    )
    
    # Add brain surface for female plot
    fig.add_trace(
        go.Surface(
            x=x_brain, y=y_brain, z=z_brain,
            opacity=0.8,
            colorscale='Greys',
            showscale=False,
            name='Brain Surface'
        ),
        row=1, col=2
    )
    
    # Add EEG points for female plot
    fig.add_trace(
        go.Scatter3d(
            x=x_eeg, y=y_eeg, z=z_eeg,
            mode='markers+text',
            marker=dict(
                size=10,
                color=female_power,
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(
                    title='Power (μV²)',
                    x=1.0
                )
            ),
            text=ch_names,
            textposition="top center",
            name='Female EEG'
        ),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        title=dict(
            text='3D Brain Activity Visualization During Financial Decision Making',
            x=0.5,
            y=0.95,
            xanchor='center',
            yanchor='top',
            font=dict(size=20)
        ),
        width=1500,
        height=800,
        scene=dict(
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        scene2=dict(
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        )
    )
    
    return fig

def save_3d_plots(processed_data, output_dir='eeg_visualizations'):
    """
    Save 3D brain plots to HTML file.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Create and save 3D plots
    fig_3d = create_3d_brain_plots(processed_data)
    output_path = os.path.join(output_dir, '3d_brain_comparison.html')
    fig_3d.write_html(output_path)
    
    print(f"3D brain visualization saved to: {output_path}")

# Main execution
if __name__ == "__main__":
    
    # Define paths to your existing data
    data_dir = "eeg_data"  # your folder containing the EEG data files
    metadata_path = "eeg_data/metadata.csv"  # path to your metadata file
       
    # Load data from CSV files
    raw_objects = load_eeg_data(data_dir, metadata_path)
    print("Data loaded successfully")
    
    # Preprocess data
    processed_data = preprocess_data(raw_objects)
    print("Data preprocessing completed")
    
    # Save all plots to HTML files
    save_plots_to_html(processed_data)

    save_3d_plots(processed_data)

Creating RawArray with float64 data, n_channels=20, n_times=1000
    Range : 0 ... 999 =      0.000 ...     3.918 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1000
    Range : 0 ... 999 =      0.000 ...     3.918 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1000
    Range : 0 ... 999 =      0.000 ...     3.918 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1000
    Range : 0 ... 999 =      0.000 ...     3.918 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1000
    Range : 0 ... 999 =      0.000 ...     3.918 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1000
    Range : 0 ... 999 =      0.000 ...     3.918 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1000
    Range : 0 ... 999 =      0.000 ...     3.918 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1000
    Range : 0 ... 999 =      0.000 ...     3.918 secs

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


Fitting ICA took 0.8s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 20 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 843 samples (3.306 s)

Fitting ICA to data using 20 channels (please be patient, this may take a while)
Selecting by number: 15 components


/opt/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


Fitting ICA took 0.6s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 20 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 843 samples (3.306 s)

Fitting ICA to data using 20 channels (please be patient, this may take a while)
Selecting by number: 15 components


...

Fitting ICA took 0.6s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 20 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 843 samples (3.306 s)

Fitting ICA to data using 20 channels (please be patient, this may take a while)
Selecting by number: 15 components


/opt/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


Fitting ICA took 0.6s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 20 PCA components


/opt/anaconda3/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Data preprocessing completed
Plots have been saved to the 'eeg_visualizations' directory:
1. male_topographic.html - Male brain activity topographic map
2. female_topographic.html - Female brain activity topographic map
3. time_frequency_comparison.html - Time-frequency comparison of both groups
3D brain visualization saved to: eeg_visualizations/3d_brain_comparison.html
